In [1]:
!git clone https://github.com/Hannibal96/ImageCaptionProject.git

Cloning into 'ImageCaptionProject'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 202 (delta 116), reused 100 (delta 37), pack-reused 0
Receiving objects: 100% (202/202), 8.90 MiB | 23.07 MiB/s, done.
Resolving deltas: 100% (116/116), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd ImageCaptionProject/

/content/ImageCaptionProject


In [4]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


In [5]:
from data import *

In [6]:
from models import *

In [16]:
from data import *
from models import *
import torchvision.transforms as T
from torchtext.vocab import GloVe # for pretrained model
import torch.optim as optim
from torch.utils.data import DataLoader
import pickle
from nltk.translate.bleu_score import sentence_bleu
from tqdm import tqdm

def save_model(model, num_epochs):
    path = "caption_model_E_"+str(num_epochs)+".torch"
    torch.save(model, path)


def evaluate(model, val_data_set):
    blew_score = 0
    total_val_loss = 0

    with torch.no_grad():
        for idx, (image, captions) in tqdm(enumerate(iter(val_data_set))):
            image, captions = image.to(device), captions.to(device)

            all_captions = val_dataset.get_last_captions()
            features = model.encoder(image[0:1].to(device))
            caps, alphas = model.decoder.generate_caption(features, vocab=vocab)
            hyp_caption = ' '.join(caps)
            curr_blew_score = sentence_bleu(references=all_captions, hypothesis=hyp_caption.split())
            blew_score += curr_blew_score

            outputs, attentions = model(image, captions)
            outputs = outputs.to(device)
            targets = captions[:, 1:]
            targets = targets.to(device)
            val_loss = criterion(outputs.view(-1, vocab_size), targets.reshape(-1))
            total_val_loss += val_loss.item()

    perplexity = total_val_loss / len(val_data_set)
    perplexity = np.exp(perplexity)
    return blew_score / len(val_data_set), total_val_loss/len(val_data_set), perplexity



In [ ]:
!ls 

In [12]:


images_path = '/content/drive/MyDrive/ImageCaption/Images'
captions_file_path = "captions.txt"
karpathy_json_path = 'Karpathy_data.json'

# define the transforms to be applied which needed for the pretrained CNN
transforms = T.Compose([
    T.Resize(226),
    T.RandomCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

#build vocab
vocab = build_vocab(captions_file_path=captions_file_path)

#build datasets
train, val, test = karpathy_split(captions_file_path, karpathy_json_path)
train_dataset = FlickrDataset(root_dir=images_path,vocab= vocab, captions_df=train,transform=transforms)
val_dataset = FlickrDataset(root_dir=images_path,vocab= vocab, captions_df=val,transform=transforms)
test_dataset = FlickrDataset(root_dir=images_path,vocab= vocab, captions_df=test,transform=transforms)
print("Finished building the Datasets.")

# Hyperparams
weights_matrix = None
# load pretrained embeddings (to train embeddings from scrach just set the embed_size)
#g = GloVe(name ='6B', dim=100)
embed_size = 300#g.dim
#weights_matrix = load_embedding_weights(vocab, g)

vocab_size = len(vocab)
attention_dim = 256
encoder_dim = 2048
decoder_dim = 512
learning_rate = 3e-4
BATCH_SIZE = 32

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# init model
model = EncoderDecoder(
    embed_size=embed_size,
    vocab_size=vocab_size,
    attention_dim=attention_dim,
    encoder_dim=encoder_dim,
    decoder_dim=decoder_dim,
    embedding_weights = weights_matrix
).to(device)

pad_idx = vocab.stoi["<PAD>"]
train_data = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, num_workers=0,
                        shuffle=True, collate_fn=CapsCollate(pad_idx=pad_idx, batch_first=True))
val_data = DataLoader(dataset=val_dataset, batch_size=1, num_workers=0,
                      shuffle=True, collate_fn=CapsCollate(pad_idx=pad_idx, batch_first=True))

criterion = nn.CrossEntropyLoss(ignore_index=vocab.stoi["<PAD>"]).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

num_epochs = 10
print_every = 1000


100%|██████████| 40455/40455 [00:02<00:00, 18195.76it/s]


Finished building the Datasets.


In [20]:
def train_one_epoch(model, train_data):
  total_loss = 0
  for idx, (image, captions) in tqdm(enumerate(iter(train_data))):
      image, captions = image.to(device), captions.to(device)

      # Zero the gradients.
      optimizer.zero_grad()

      # Feed forward
      outputs, attentions = model(image, captions)
      outputs = outputs.to(device)

      # Calculate the batch loss.
      targets = captions[:, 1:]
      targets = targets.to(device)
      loss = criterion(outputs.view(-1, vocab_size), targets.reshape(-1))

      # Backward pass.
      loss.backward()

      # Update the parameters in the optimizer.
      optimizer.step()

      total_loss += loss.item()
    
  return total_loss/len(train_data)

In [17]:
len(train_data)

938

In [ ]:
loss_list = []
perplexity_list = []
bleu_list = []
total_loss = 0
for epoch in range(1, num_epochs + 1):
    model.train()
    avg_training_loss = train_one_epoch(model, train_data)
    print('average loss = {}'.format(avg_training_loss))
    save_model(model, epoch)

    model.eval()
    bleu, loss, perp = evaluate(model, val_data)
    print("Epoch: {} loss: {:.3f}, perplexity: {:.3f}, BLEU: {:.3f}".format(epoch, loss, perp, bleu))
    perplexity_list.append(perp)
    loss_list.append(loss)
    bleu_list.append(bleu)

    pickle.dump(perplexity_list, open('perplexity_list.p', 'wb'))
    pickle.dump(loss_list, open('loss_list.p', 'wb'))
    pickle.dump(bleu_list, open('blew_list.p', 'wb'))


plt.plot(loss_list)
plt.title('Loss')
plt.show()

plt.plot(perplexity_list)
plt.title('Perplexity')
plt.show()

plt.plot(bleu_list)
plt.title('BLEU')
plt.show()




0it [00:00, ?it/s]


1it [00:08,  8.04s/it]


2it [00:16,  8.19s/it]


3it [00:25,  8.28s/it]


4it [00:32,  8.17s/it]


5it [00:40,  7.93s/it]


6it [00:48,  7.86s/it]


7it [00:55,  7.67s/it]


8it [01:02,  7.46s/it]


9it [01:09,  7.31s/it]


10it [01:16,  7.39s/it]


11it [01:24,  7.44s/it]


12it [01:32,  7.54s/it]


13it [01:39,  7.45s/it]


14it [01:46,  7.25s/it]


15it [01:52,  7.12s/it]


16it [02:02,  7.74s/it]


17it [02:09,  7.59s/it]


18it [02:16,  7.40s/it]


19it [02:23,  7.34s/it]


20it [02:30,  7.25s/it]


21it [02:38,  7.54s/it]


22it [02:45,  7.42s/it]


23it [02:52,  7.09s/it]


24it [02:59,  7.15s/it]


25it [03:05,  6.90s/it]


26it [03:12,  6.73s/it]


27it [03:19,  6.81s/it]


28it [03:25,  6.76s/it]


29it [03:32,  6.61s/it]


30it [03:38,  6.59s/it]


31it [03:45,  6.68s/it]


32it [03:51,  6.61s/it]


33it [03:58,  6.52s/it]


34it [04:04,  6.51s/it]


35it [04:10,  6.39s/it]


36it [04:16,  6.10s/it]


37it [04:22,  6.21s/it]


38it [04:29,  6.50s/it]

average loss = 3.799947449393364


Streaming output truncated to the last 5000 lines.
1433it [01:53, 17.36it/s]


1436it [01:53, 18.41it/s]


1438it [01:53, 18.58it/s]


1441it [01:53, 19.26it/s]


1443it [01:53, 14.03it/s]


1445it [01:54, 14.91it/s]


1447it [01:54, 15.84it/s]


1449it [01:54, 16.48it/s]


1451it [01:54, 17.16it/s]


1453it [01:54, 17.89it/s]


1455it [01:54, 13.27it/s]


1457it [01:54, 14.45it/s]


1459it [01:54, 15.45it/s]


1461it [01:54, 16.47it/s]


1463it [01:55, 16.95it/s]


1465it [01:55, 17.39it/s]


1467it [01:55, 17.98it/s]


1469it [01:55, 18.43it/s]


1471it [01:55, 18.59it/s]


1473it [01:55, 18.72it/s]


1475it [01:55, 18.52it/s]


1477it [01:55, 18.37it/s]


1479it [01:55, 18.35it/s]


1481it [01:56, 18.09it/s]


1483it [01:56, 17.86it/s]


1485it [01:56, 17.96it/s]


1487it [01:56, 17.90it/s]


1489it [01:56, 18.03it/s]


1492it [01:56, 18.93it/s]


1494it [01:56, 18.56it/s]


1497it [01:56, 19.18it/s]


1499it [01:57, 18.66it/s]


1501it [01:57, 18.90it/s]


1503it [01:57, 19.03it/s]

Epoch: 1 loss: 3.269, perplexity: 26.274, BLEU: 0.462





1it [00:00,  1.72it/s]


2it [00:01,  1.73it/s]


3it [00:01,  1.72it/s]


4it [00:02,  1.72it/s]


5it [00:02,  1.74it/s]


6it [00:03,  1.73it/s]


7it [00:04,  1.75it/s]


8it [00:04,  1.74it/s]


9it [00:05,  1.73it/s]


10it [00:05,  1.72it/s]


11it [00:06,  1.74it/s]


12it [00:06,  1.74it/s]


13it [00:07,  1.77it/s]


14it [00:08,  1.74it/s]


15it [00:08,  1.75it/s]


16it [00:09,  1.74it/s]


17it [00:09,  1.73it/s]


18it [00:10,  1.74it/s]


19it [00:10,  1.74it/s]


20it [00:11,  1.73it/s]


21it [00:12,  1.74it/s]


22it [00:12,  1.74it/s]


23it [00:13,  1.75it/s]


24it [00:13,  1.76it/s]


25it [00:14,  1.77it/s]


26it [00:14,  1.75it/s]


27it [00:15,  1.76it/s]


28it [00:16,  1.78it/s]


29it [00:16,  1.76it/s]


30it [00:17,  1.76it/s]


31it [00:17,  1.77it/s]


32it [00:18,  1.77it/s]


33it [00:18,  1.78it/s]


34it [00:19,  1.80it/s]


35it [00:19,  1.80it/s]


36it [00:20,  1.81it/s]


37it [00:21,  1.79it/s]


38it [00:21,  1.79it/s]


39it [00:22,  1.75

...

Streaming output truncated to the last 5000 lines.
1598it [01:26, 19.25it/s]


1600it [01:26, 18.26it/s]


1602it [01:26, 17.78it/s]


1604it [01:26, 17.95it/s]


1606it [01:26, 17.62it/s]


1608it [01:27, 17.80it/s]


1610it [01:27, 17.81it/s]


1612it [01:27, 18.07it/s]


1614it [01:27, 18.25it/s]


1616it [01:27, 17.98it/s]


1618it [01:27, 18.04it/s]


1620it [01:27, 17.85it/s]


1622it [01:27, 17.19it/s]


1624it [01:27, 17.52it/s]


1626it [01:28, 18.10it/s]


1628it [01:28, 18.47it/s]


1630it [01:28, 18.22it/s]


1632it [01:28, 18.34it/s]


1634it [01:28, 18.44it/s]


1636it [01:28, 18.72it/s]


1638it [01:28, 18.90it/s]


1640it [01:28, 18.85it/s]


1642it [01:28, 18.81it/s]


1644it [01:29, 19.13it/s]


1647it [01:29, 19.83it/s]


1649it [01:29, 19.81it/s]


1651it [01:29, 19.32it/s]


1653it [01:29, 18.48it/s]


1655it [01:29, 18.30it/s]


1657it [01:29, 18.46it/s]


1659it [01:29, 18.83it/s]


1662it [01:29, 19.71it/s]


1664it [01:30, 19.58it/s]


1666it [01:30, 19.10it/s]

Epoch: 3 loss: 2.858, perplexity: 17.418, BLEU: 0.473





1it [00:00,  1.75it/s]


2it [00:01,  1.70it/s]


3it [00:01,  1.70it/s]


4it [00:02,  1.71it/s]


5it [00:02,  1.70it/s]


6it [00:03,  1.70it/s]


7it [00:04,  1.72it/s]


8it [00:04,  1.69it/s]


9it [00:05,  1.71it/s]


10it [00:05,  1.69it/s]


11it [00:06,  1.73it/s]


12it [00:07,  1.72it/s]


13it [00:07,  1.72it/s]


14it [00:08,  1.73it/s]


15it [00:08,  1.72it/s]


16it [00:09,  1.71it/s]


17it [00:09,  1.70it/s]


18it [00:10,  1.68it/s]


19it [00:11,  1.68it/s]


20it [00:11,  1.71it/s]


21it [00:12,  1.73it/s]


22it [00:12,  1.72it/s]


23it [00:13,  1.73it/s]


24it [00:14,  1.72it/s]


25it [00:14,  1.72it/s]


26it [00:15,  1.72it/s]


27it [00:15,  1.72it/s]


28it [00:16,  1.70it/s]


29it [00:16,  1.72it/s]


30it [00:17,  1.73it/s]


31it [00:18,  1.69it/s]


32it [00:18,  1.71it/s]


33it [00:19,  1.70it/s]


34it [00:19,  1.70it/s]


35it [00:20,  1.73it/s]


36it [00:21,  1.74it/s]


37it [00:21,  1.73it/s]


38it [00:22,  1.69it/s]


39it [00:22,  1.71

average loss = 2.692328441371796





0it [00:00, ?it/s]


2it [00:00, 14.19it/s]


4it [00:00, 14.78it/s]


6it [00:00, 15.61it/s]


8it [00:00, 16.20it/s]


10it [00:00, 16.51it/s]


12it [00:00, 16.34it/s]


14it [00:00, 17.11it/s]


16it [00:00, 17.25it/s]


18it [00:01, 17.07it/s]


20it [00:01, 17.60it/s]


22it [00:01, 17.32it/s]


24it [00:01, 17.56it/s]


26it [00:01, 17.69it/s]


28it [00:01, 17.62it/s]


30it [00:01, 17.83it/s]


32it [00:01, 18.07it/s]


34it [00:01, 18.12it/s]


36it [00:02, 17.64it/s]


38it [00:02, 17.39it/s]


40it [00:02, 17.48it/s]


42it [00:02, 17.42it/s]


45it [00:02, 18.56it/s]


47it [00:02, 18.72it/s]


49it [00:02, 19.04it/s]


51it [00:02, 18.62it/s]


53it [00:02, 18.71it/s]


55it [00:03, 18.55it/s]


57it [00:03, 17.88it/s]


59it [00:03, 17.24it/s]


61it [00:03, 16.78it/s]


63it [00:03, 17.15it/s]


65it [00:03, 17.23it/s]


67it [00:03, 17.66it/s]


69it [00:03, 17.88it/s]


71it [00:04, 18.12it/s]


73it [00:04, 18.00it/s]


75it [00:04, 18.16it/s]


77it [00:04, 17.99